<div style="position: relative;">
<img src="https://user-images.githubusercontent.com/7065401/98728503-5ab82f80-2378-11eb-9c79-adeb308fc647.png"></img>

<h1 style="color: white; position: absolute; top:27%; left:10%;">
     Advanced Python
</h1>
<h2 style="color: white; position: absolute; top:36%; left:10%;">
    Iterators, Generators, Context Managers, and Decorators
</h2>


<h3 style="color: #ef7d22; font-weight: normal; position: absolute; top:58%; left:10%;">
    David Mertz, Ph.D.
</h3>

<h3 style="color: #ef7d22; font-weight: normal; position: absolute; top:63%; left:10%;">
    Data Scientist
</h3>
</div>

# More Itertools

In this lesson we continue to explore combining iterators, both using additional capabilities in `itertools`, and a few extras utilizing the 3rd party `more-itertools` module that adds additional useful function.  There are a number of short "recipes" in the official documentation of `itertools`—all of these recipes, along with numerous other functions, are included in `more_itertools`.

In [1]:
from itertools import *
from more_itertools import *

## Run-length encoding

Sometimes data can be compressed very time- and memory-efficiently using a technique called *run-length* encoding.  The idea is that certain data sets, the same element may occur numerous times successively.  If so, it can be more compact to represent that as a number counting occurrences, followed by the element that is repeated.  As compression techniques go, this is rarely the most compressed result, although often RLE is combined with other techniques.  Let us implement it in an iterator style to show some of the conciseness of `itertools`.

As sample data, let us look at a FASTA record of rhibosomal RNA.  If the recurring objects were larger than a single character, RLE would be more significant, but this suffices to demonstrate the concept.  Here is sample sequence:

In [2]:
ab000482 = """
agtttgatcctggctcagaacgaacgctggcggcaggcctaacacatgcaagtcgaggga
gaagctatcttcggatnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn
nnnnnnnnnnnnnnaaacgactgctaataccgcatacgcccttcgggggaaagatttatc
gctattcgattggcccgcgttagattagctaagttggtaaggtaacggcttaccaaggcg
acgatctatagctggtttgagaggatgatcagccacactgggactgagacacggcccaga
ctcctacgggaggcagcagtggggaatattgcgcaatggaggaaactctgacgcagccat
gccgcgtgagtgaagaaggccttagggttgtaaagctctttcagacgtgatgaatgatga
cagtagcgtcaaaagaagttccggctaaacttcgtgccagcagccgcggtaatacgaagg
gaactagcgttgttcggatttactgggcgtaaagagcatgtaggcggattggacagttga
gggtgaaatcccagagctcaactctggaacggccttcaatacttccagtctagagtccgt
aagggggtggtggaattccgagtgtagaggtgaaattcgtagatattcggaggaacacca
gtggcgaaggcgaccacctggtacggtactgacgctgagatgcgaaagcgtggggagcaa
acaggattagataccctggtagtccacgccgtaaacgatgagtgctagttgtcaggatgt
ttacatcttggtgacgcagctaacgcattaagcactccgcctggggagtacggtcgcaag
attaaaactcaaaggaattgacgggggcccgcacaagcggtggagcatgtggtttaattc
gaagcaacgcgaagaaccttaccaattcttgacatacctgtcgcgatttccagagatgga
tttcttcagttcggctggacaggatacaggtgctgcatggctgtcgtcagctcgtgtcgt
gagatgttgggttaagtcccgcaacgagcgcaaccctcacccctagttgccagcatttag
ttgggcactctatgggaactgccggtgacaagccggaggaaggtggggatgacgtcaagt
catcatggcccttacggattgggctacacacgtgctacaatggtaactacagtgggcagc
gacgtcgcgaggcgaagcaaatctccaaaagttatctcagttcggattgttctctgcaac
tcgagagcatgaagtcggaatcgcctagtaatcgcggatcagcatgccgcggtgaata
"""

Let us wrap that slightly to remove the the optional newlines.  This rRNA sample is quite small, but the genes in human DNA, for example are vastly larger (but still of this general form in FASTA format).  An iterator over individual nucleic acids can be constructed as:

In [3]:
def symbols(fasta):
    # Use a generator comprehension
    yield from (c for c in fasta if c != '\n')

# A few symbols from first line continuing on second
list(islice(symbols(ab000482), 55, 65))

['a', 'g', 'g', 'g', 'a', 'g', 'a', 'a', 'g', 'c']

A generic RLE encoder can be constructed using `itertools.groupby()` which this thinly wraps.  Notice that we encode incrementally as symbols are read.

In [4]:
def rle_encode(it):
    for k, g in groupby(it):
        yield (k, len(list(g)))

In [5]:
seq = symbols(ab000482)
for enc in rle_encode(islice(seq, 70, 140)):
    print(enc, end=" ")

('t', 1) ('c', 1) ('g', 2) ('a', 1) ('t', 1) ('n', 58) ('a', 3) ('c', 1) ('g', 1) ('a', 1) 

We can reverse the encoding be implementing a similarly short function using `itertools.repeat()` and `itertools.chain.from_iterable()`.  Repeating is just yielding the same items back numerous times.  Chaining is interesting.  It allows the lazy combination of as many iterables as you like, the next as soon as the previous one is exhausted.  However, if what you want to chain is not only a handful of named iterables, but rather an iterable of perhaps thousands of millions of iterables, it is impractical to pass these all as explicit arguments.  For example:

In [6]:
a = {1, 2, 3}
b = "ABC"
c = [9, 4, 2]
for x in chain(a, b, c):
    print(x, end=' ')

1 2 3 A B C 9 4 2 

In [7]:
def iter_of_iters():
    yield a
    yield b
    yield c
    
d = iter_of_iters()
    
for x in chain.from_iterable(d):
    print(x, end=' ')

1 2 3 A B C 9 4 2 

So we can decode incrementally, taking each tuple `(val, ncount)` that we get from RLE encoding.

In [8]:
def rle_decode(it):
    yield from chain.from_iterable(repeat(x, n) for x, n in it)

In [9]:
seq = symbols(ab000482)
encoded = rle_encode(seq)
decoded = rle_decode(encoded)
print(seq, encoded, decoded, sep='\n')

<generator object symbols at 0x7f7f600a7040>
<generator object rle_encode at 0x7f7f6014f2e0>
<generator object rle_decode at 0x7f7f60114f90>


Looping through decoded should reproduce our original sequence.

In [10]:
for symbol in islice(decoded, 60):
    print(symbol, end='')

agtttgatcctggctcagaacgaacgctggcggcaggcctaacacatgcaagtcgaggga

## Combining iterables

Let's build further on the idea of chaining that we saw used in `rle_decode()`.  FASTA files consist of a header line followed by sequence information.  We might have stored millions or billions of these, and wish to process them only incrementally.  For our stipulated purpose, we would like not to read in all at once, nor ever to read in files after a search had found a certain pattern.

Combining things we've done, let us identify the first sequence of more than 50 repetitions of the same symbol across a family of separate FASTA files.

In [11]:
from glob import glob
from collections import namedtuple
Record = namedtuple("Record", "metadata sequence")

def read_fasta(pat):
    for fname in glob(pat):
        with open(fname) as fasta:
            # Return a pair of sequence description and seq as iterator
            yield Record(next(fasta).strip(), symbols(fasta.read()))
            
next(read_fasta("rRNA*.fasta"))

Record(metadata='>AB000393_1|Vibrio halioticoli|16S rRNA', sequence=<generator object symbols at 0x7f7f600fcf20>)

Let us find those sequences with 30 repeated symbols.

In [12]:
for record in read_fasta("rRNA*.fasta"):
    long_subseq = [enc for enc in rle_encode(record.sequence) if enc[1] >= 30]
    if long_subseq:
        print(record.metadata)
        print(*long_subseq)

>AB000476_1|Novispirillum itersonii|16S rRNA
('n', 34)
>AB000478_1|Novispirillum itersonii|16S rRNA
('n', 34)
>AB000477_1|Novispirillum itersonii|16S rRNA
('n', 34)
>AB000482_1|Terasakiella pusilla|16S rRNA
('n', 58)
>AB000481_1|Aquaspirillum polymorphum|16S rRNA
('n', 99)


## Interleaving, tupling, chaining

The functions `itertools.chain()` and `itertools.chain.from_iterable()` combine multiple iterables.  Built-in `zip()` and `itertools.zip_longest()` also do this, although in a manner that incrementally advances each iterables.  These are all useful, but sometimes a slightly different way of combining is useful instead.  `more_itertools` provides `interleave()` and `interleave_longest()`.  Let us look at all these options using some some simple collections (iterables).  

These all work with iterables in general, including infinite ones. But it is easier to see with small collections.

In [13]:
a = {1, 2, 3, 4, 5}
b = "ABCD"
c = [99, 88, 77]

In [14]:
list(chain(a, b, c))

[1, 2, 3, 4, 5, 'A', 'B', 'C', 'D', 99, 88, 77]

In [15]:
list(chain.from_iterable([a, b, c]))

[1, 2, 3, 4, 5, 'A', 'B', 'C', 'D', 99, 88, 77]

In [16]:
list(zip(a, b, c))

[(1, 'A', 99), (2, 'B', 88), (3, 'C', 77)]

In [17]:
list(zip_longest(a, b, c))

[(1, 'A', 99), (2, 'B', 88), (3, 'C', 77), (4, 'D', None), (5, None, None)]

In [18]:
list(interleave(a, b, c))

[1, 'A', 99, 2, 'B', 88, 3, 'C', 77]

In [19]:
list(interleave_longest(a, b, c))

[1, 'A', 99, 2, 'B', 88, 3, 'C', 77, 4, 'D', 5]

## Combinatorics

One of the useful things `itertools` can do is combinatorics on iterators.  In the standard library, this consists of `permutations()`, `combinations()`, Cartesian `product()`, and `combinations_with_replacement()`.  But `more_itertools()` adds a bunch more including (but not limited to) `distinct_permutations()`, `circular_shifts()`, `partitions()`, `set_partitions()`, and `powerset()`.

The functions generally consume entire iterators as implemented, and are not suitable for infinite iterators. However, for finite iterators, they provide efficient and *lazy* ways to get arrangements of the source iteration elements.

In [20]:
a = {1, 2, 3, 4}
b = "ABC"
c = [99, 88]

One convenience of Cartesian product is that it can sometimes simplify nested loops.

In [21]:
for prod in product(a, b):
    print(prod, end=" ")

(1, 'A') (1, 'B') (1, 'C') (2, 'A') (2, 'B') (2, 'C') (3, 'A') (3, 'B') (3, 'C') (4, 'A') (4, 'B') (4, 'C') 

In [22]:
# Save two nested elements while looking at all combos
for i, j, k in product(a, b, c):
    print(i, j, k, sep='', end=' ')

1A99 1A88 1B99 1B88 1C99 1C88 2A99 2A88 2B99 2B88 2C99 2C88 3A99 3A88 3B99 3B88 3C99 3C88 4A99 4A88 4B99 4B88 4C99 4C88 

In [23]:
list(permutations(b))

[('A', 'B', 'C'),
 ('A', 'C', 'B'),
 ('B', 'A', 'C'),
 ('B', 'C', 'A'),
 ('C', 'A', 'B'),
 ('C', 'B', 'A')]

In [24]:
list(permutations(b, r=2))

[('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'C'), ('C', 'A'), ('C', 'B')]

In [25]:
# Not order dependent for distinct combination
list(combinations(b, r=2))

[('A', 'B'), ('A', 'C'), ('B', 'C')]

In [26]:
print(list(powerset(a)))

[(), (1,), (2,), (3,), (4,), (1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4), (1, 2, 3), (1, 2, 4), (1, 3, 4), (2, 3, 4), (1, 2, 3, 4)]


Sometimes you worry about distinctness.  The standard tools only consider distinctness relative to position in an iterator.  Some extra in `more_itertools` also filter by item equality.

In [27]:
three_letters = list(permutations('david', r=3))
print(len(three_letters))
three_letters

60


[('d', 'a', 'v'),
 ('d', 'a', 'i'),
 ('d', 'a', 'd'),
 ('d', 'v', 'a'),
 ('d', 'v', 'i'),
 ('d', 'v', 'd'),
 ('d', 'i', 'a'),
 ('d', 'i', 'v'),
 ('d', 'i', 'd'),
 ('d', 'd', 'a'),
 ('d', 'd', 'v'),
 ('d', 'd', 'i'),
 ('a', 'd', 'v'),
 ('a', 'd', 'i'),
 ('a', 'd', 'd'),
 ('a', 'v', 'd'),
 ('a', 'v', 'i'),
 ('a', 'v', 'd'),
 ('a', 'i', 'd'),
 ('a', 'i', 'v'),
 ('a', 'i', 'd'),
 ('a', 'd', 'd'),
 ('a', 'd', 'v'),
 ('a', 'd', 'i'),
 ('v', 'd', 'a'),
 ('v', 'd', 'i'),
 ('v', 'd', 'd'),
 ('v', 'a', 'd'),
 ('v', 'a', 'i'),
 ('v', 'a', 'd'),
 ('v', 'i', 'd'),
 ('v', 'i', 'a'),
 ('v', 'i', 'd'),
 ('v', 'd', 'd'),
 ('v', 'd', 'a'),
 ('v', 'd', 'i'),
 ('i', 'd', 'a'),
 ('i', 'd', 'v'),
 ('i', 'd', 'd'),
 ('i', 'a', 'd'),
 ('i', 'a', 'v'),
 ('i', 'a', 'd'),
 ('i', 'v', 'd'),
 ('i', 'v', 'a'),
 ('i', 'v', 'd'),
 ('i', 'd', 'd'),
 ('i', 'd', 'a'),
 ('i', 'd', 'v'),
 ('d', 'd', 'a'),
 ('d', 'd', 'v'),
 ('d', 'd', 'i'),
 ('d', 'a', 'd'),
 ('d', 'a', 'v'),
 ('d', 'a', 'i'),
 ('d', 'v', 'd'),
 ('d', 'v'

In [28]:
three_distinct = list(distinct_permutations('david', r=3))
print(len(three_distinct))
three_distinct

33


[('a', 'd', 'd'),
 ('a', 'd', 'i'),
 ('a', 'd', 'v'),
 ('a', 'i', 'd'),
 ('a', 'i', 'v'),
 ('a', 'v', 'd'),
 ('a', 'v', 'i'),
 ('d', 'a', 'd'),
 ('d', 'a', 'i'),
 ('d', 'a', 'v'),
 ('d', 'd', 'a'),
 ('d', 'd', 'i'),
 ('d', 'd', 'v'),
 ('d', 'i', 'a'),
 ('d', 'i', 'd'),
 ('d', 'i', 'v'),
 ('d', 'v', 'a'),
 ('d', 'v', 'd'),
 ('d', 'v', 'i'),
 ('i', 'a', 'd'),
 ('i', 'a', 'v'),
 ('i', 'd', 'a'),
 ('i', 'd', 'd'),
 ('i', 'd', 'v'),
 ('i', 'v', 'a'),
 ('i', 'v', 'd'),
 ('v', 'a', 'd'),
 ('v', 'a', 'i'),
 ('v', 'd', 'a'),
 ('v', 'd', 'd'),
 ('v', 'd', 'i'),
 ('v', 'i', 'a'),
 ('v', 'i', 'd')]

Partitions can also be handy at times (from `more_itertools`):

In [29]:
for part in partitions('mertz'):
    for segment in part:
        print(''.join(segment), end=' : ')
    print()

mertz : 
m : ertz : 
me : rtz : 
mer : tz : 
mert : z : 
m : e : rtz : 
m : er : tz : 
m : ert : z : 
me : r : tz : 
me : rt : z : 
mer : t : z : 
m : e : r : tz : 
m : e : rt : z : 
m : er : t : z : 
me : r : t : z : 
m : e : r : t : z : 


Simple partitions are order preserving, but set partitions can rearrange before partitioning (and hence there are many more).

In [30]:
for part in set_partitions('mertz'):
    for segment in part:
        print(''.join(segment), end=' : ')
    print()

mertz : 
m : ertz : 
me : rtz : 
e : mrtz : 
mer : tz : 
er : mtz : 
mr : etz : 
r : metz : 
mert : z : 
ert : mz : 
mrt : ez : 
rt : mez : 
met : rz : 
et : mrz : 
mt : erz : 
t : merz : 
m : e : rtz : 
m : er : tz : 
m : r : etz : 
m : ert : z : 
m : rt : ez : 
m : et : rz : 
m : t : erz : 
me : r : tz : 
e : mr : tz : 
e : r : mtz : 
me : rt : z : 
e : mrt : z : 
e : rt : mz : 
me : t : rz : 
e : mt : rz : 
e : t : mrz : 
mer : t : z : 
er : mt : z : 
er : t : mz : 
mr : et : z : 
r : met : z : 
r : et : mz : 
mr : t : ez : 
r : mt : ez : 
r : t : mez : 
m : e : r : tz : 
m : e : rt : z : 
m : e : t : rz : 
m : er : t : z : 
m : r : et : z : 
m : r : t : ez : 
me : r : t : z : 
e : mr : t : z : 
e : r : mt : z : 
e : r : t : mz : 
m : e : r : t : z : 
